In [1]:
from sdim import *
import cirq # using cirq to visualize the circuit
import numpy as np

### Encoding
We encode one qubit of information distributed amongst three using CNOT's. This is only protected against a single X error.

In [2]:
# Define the circuit
circuit = Circuit(5, 2) # Three qubit repitition code with two ancilla qubits
circuit.add_gate('CNOT', 0, 1)
circuit.add_gate('CNOT', 0, 2)

Circuit(num_qudits=5, dimension=2, operations=[CircuitInstruction(gate_data=GateData(gateMap={'I': Gate(name='I', arg_count=1, gate_id=0), 'X': Gate(name='X', arg_count=1, gate_id=1), 'X_INV': Gate(name='X_INV', arg_count=1, gate_id=2), 'Z': Gate(name='Z', arg_count=1, gate_id=3), 'Z_INV': Gate(name='Z_INV', arg_count=1, gate_id=4), 'H': Gate(name='H', arg_count=1, gate_id=5), 'H_INV': Gate(name='H_INV', arg_count=1, gate_id=6), 'P': Gate(name='P', arg_count=1, gate_id=7), 'P_INV': Gate(name='P_INV', arg_count=1, gate_id=8), 'CNOT': Gate(name='CNOT', arg_count=2, gate_id=9), 'CNOT_INV': Gate(name='CNOT_INV', arg_count=2, gate_id=10), 'CZ': Gate(name='CZ', arg_count=2, gate_id=11), 'CZ_INV': Gate(name='CZ_INV', arg_count=2, gate_id=12), 'SWAP': Gate(name='SWAP', arg_count=2, gate_id=13), 'M': Gate(name='M', arg_count=1, gate_id=14), 'M_X': Gate(name='M_X', arg_count=1, gate_id=15)}, aliasMap={'R': 'H', 'DFT': 'H', 'R_INV': 'H_INV', 'DFT_INV': 'H_INV', 'H_DAG': 'H_INV', 'R_DAG': 'H_INV',

In [3]:
def insert_x_error(circuit, p=0.5):
    if np.random.rand() < p:
        i = np.random.choice(3)  # choose a random qubit from 0, 1, 2
        circuit.add_gate('X', i)
        print(f'Error inserted on qubit {i}')
    else:
        print('No error inserted')
    return
insert_x_error(circuit, p=0.9)

Error inserted on qubit 1


### Syndrome Measurements
We check the operators $Z\otimes Z \otimes I$ and $I\otimes Z \otimes Z$, but these are the generators and so other syndromes can be measured, leading to different circuit constructions.

In [4]:
# ZZI
circuit.add_gate('CNOT', 0, 3)
circuit.add_gate('CNOT', 1, 3)
# IZZ
circuit.add_gate('CNOT', 1, 4)
circuit.add_gate('CNOT', 2, 4)
# Measure
circuit.add_gate('MEASURE', 3)
circuit.add_gate('MEASURE', 4)

program = Program(circuit)
result = program.simulate()
print(result)

[Measured qudit (3) as (1) and was deterministic, Measured qudit (4) as (1) and was deterministic]


In [5]:
cirq_circuit = circuit_to_cirq_circuit(circuit)
print(cirq_circuit)

                             ┌─────────────────┐                                     ┌─────────────────┐
0 (d=2): ───CNOT_2_control────CNOT_2_control───────CNOT_2_control─────────────────────────────────────────────────────────────────
            │                 │                    │
1 (d=2): ───CNOT_2_target─────┼─────────────X_2────┼────────────────CNOT_2_control────CNOT_2_control──────────────────────────────
                              │                    │                │                 │
2 (d=2): ─────────────────────CNOT_2_target────────┼────────────────┼─────────────────┼────────────────────CNOT_2_control─────────
                                                   │                │                 │                    │
3 (d=2): ──────────────────────────────────────────CNOT_2_target────CNOT_2_target─────┼─────────────I_2────┼──────────────────────
                                                                                      │                    │
4 (d=2): ───

The following table is the syndrome and the corresponding error
\begin{array}{|c|c|}
\hline
\text{Syndrome $(q_3, q_4)$} & \text{Error} \\
\hline
00 & \text{None} \\
01 & X \text{ on qubit 2} \\
10 & X \text{ on qubit 0} \\
11 & X \text{ on qubit 1} \\
\hline
\end{array}

### Moving to qutrits
The concept is the same, measuring syndromes to calculate the parity. The new syndrome mesaurements are $Z\otimes Z^{-1} \otimes I$ and $I \otimes Z \otimes Z^{-1}$. These syndromes also have to detect more types of 'bit' flips, namely $X$ and $X^2$, where the ternary Pauli operator is no longer Hermitian.

In [6]:
qutrit_circuit = Circuit(5, 3)
qutrit_circuit.add_gate('CNOT', 0, 1) # Called CNOT but turns into SUM gate
qutrit_circuit.add_gate('CNOT', 0, 2) 

Circuit(num_qudits=5, dimension=3, operations=[CircuitInstruction(gate_data=GateData(gateMap={'I': Gate(name='I', arg_count=1, gate_id=0), 'X': Gate(name='X', arg_count=1, gate_id=1), 'X_INV': Gate(name='X_INV', arg_count=1, gate_id=2), 'Z': Gate(name='Z', arg_count=1, gate_id=3), 'Z_INV': Gate(name='Z_INV', arg_count=1, gate_id=4), 'H': Gate(name='H', arg_count=1, gate_id=5), 'H_INV': Gate(name='H_INV', arg_count=1, gate_id=6), 'P': Gate(name='P', arg_count=1, gate_id=7), 'P_INV': Gate(name='P_INV', arg_count=1, gate_id=8), 'CNOT': Gate(name='CNOT', arg_count=2, gate_id=9), 'CNOT_INV': Gate(name='CNOT_INV', arg_count=2, gate_id=10), 'CZ': Gate(name='CZ', arg_count=2, gate_id=11), 'CZ_INV': Gate(name='CZ_INV', arg_count=2, gate_id=12), 'SWAP': Gate(name='SWAP', arg_count=2, gate_id=13), 'M': Gate(name='M', arg_count=1, gate_id=14), 'M_X': Gate(name='M_X', arg_count=1, gate_id=15)}, aliasMap={'R': 'H', 'DFT': 'H', 'R_INV': 'H_INV', 'DFT_INV': 'H_INV', 'H_DAG': 'H_INV', 'R_DAG': 'H_INV',

In [7]:
def insert_qutrit_x_error(circuit, p=0.5):
    if np.random.rand() < p:
        i = np.random.choice(3)  # choose a random qubit from 0, 1, 2
        error_order = np.random.choice([1, 2])  # choose between 1 and 2
        for _ in range(error_order):
            circuit.add_gate('X', i)
        print(f'X^{error_order} error inserted on qubit {i}')
    else:
        print('No error inserted')
    return
insert_qutrit_x_error(qutrit_circuit, p=0.8)

No error inserted


In [8]:
# ZZ^2I
qutrit_circuit.add_gate('CNOT', 0, 3)
qutrit_circuit.add_gate('CNOT', 1, 3)
qutrit_circuit.add_gate('CNOT', 1, 3)
# IZZ^2
qutrit_circuit.add_gate('CNOT', 1, 4)
qutrit_circuit.add_gate('CNOT', 2, 4)
qutrit_circuit.add_gate('CNOT', 2, 4)
# Measure
qutrit_circuit.add_gate('MEASURE', 3)
qutrit_circuit.add_gate('MEASURE', 4)

qutrit_program = Program(qutrit_circuit)
qutrit_result = qutrit_program.simulate()
print(qutrit_result)

[Measured qudit (3) as (0) and was deterministic, Measured qudit (4) as (0) and was deterministic]


In [9]:
cirq_qutrit_circuit = circuit_to_cirq_circuit(qutrit_circuit)
print(cirq_qutrit_circuit)

                                                                                                 ┌─────────────────┐
0 (d=3): ───CNOT_3_control───CNOT_3_control───CNOT_3_control───────────────────────────────────────────────────────────────────────────────────────────────────
            │                │                │
1 (d=3): ───CNOT_3_target────┼────────────────┼────────────────CNOT_3_control───CNOT_3_control────CNOT_3_control───────────────────────────────────────────────
                             │                │                │                │                 │
2 (d=3): ────────────────────CNOT_3_target────┼────────────────┼────────────────┼─────────────────┼────────────────────CNOT_3_control───CNOT_3_control─────────
                                              │                │                │                 │                    │                │
3 (d=3): ─────────────────────────────────────CNOT_3_target────CNOT_3_target────CNOT_3_target─────┼─────────────I_3──

\begin{array}{|c|c|}
\hline
\text{Syndrome $(q_3, q_4)$} & \text{Error} \\
\hline
00 & \text{None} \\
10 & X \text{ on qubit 0} \\
20 & X^2 \text{ on qubit 0} \\
21 & X \text{ on qubit 1} \\
12 & X^2 \text{ on qubit 1} \\
02 & X \text{ on qubit 2} \\ 
01 & X^2 \text{ on qubit 2} \\ 
\hline
\end{array}

### Moving to ququarts
Things get a little strange in composite dimensions. The syndrome mesaurements are still $Z\otimes Z^{-1} \otimes I$ and $I \otimes Z \otimes Z^{-1}$.

In [10]:
ququart_circuit = Circuit(5, 3)
ququart_circuit.add_gate('CNOT', 0, 1) # Called CNOT but turns into SUM gate
ququart_circuit.add_gate('CNOT', 0, 2) 

Circuit(num_qudits=5, dimension=3, operations=[CircuitInstruction(gate_data=GateData(gateMap={'I': Gate(name='I', arg_count=1, gate_id=0), 'X': Gate(name='X', arg_count=1, gate_id=1), 'X_INV': Gate(name='X_INV', arg_count=1, gate_id=2), 'Z': Gate(name='Z', arg_count=1, gate_id=3), 'Z_INV': Gate(name='Z_INV', arg_count=1, gate_id=4), 'H': Gate(name='H', arg_count=1, gate_id=5), 'H_INV': Gate(name='H_INV', arg_count=1, gate_id=6), 'P': Gate(name='P', arg_count=1, gate_id=7), 'P_INV': Gate(name='P_INV', arg_count=1, gate_id=8), 'CNOT': Gate(name='CNOT', arg_count=2, gate_id=9), 'CNOT_INV': Gate(name='CNOT_INV', arg_count=2, gate_id=10), 'CZ': Gate(name='CZ', arg_count=2, gate_id=11), 'CZ_INV': Gate(name='CZ_INV', arg_count=2, gate_id=12), 'SWAP': Gate(name='SWAP', arg_count=2, gate_id=13), 'M': Gate(name='M', arg_count=1, gate_id=14), 'M_X': Gate(name='M_X', arg_count=1, gate_id=15)}, aliasMap={'R': 'H', 'DFT': 'H', 'R_INV': 'H_INV', 'DFT_INV': 'H_INV', 'H_DAG': 'H_INV', 'R_DAG': 'H_INV',

In [11]:
def insert_ququart_x_error(circuit, p=0.5):
    if np.random.rand() < p:
        i = np.random.choice(3)  # choose a random qubit from 0, 1, 2
        error_order = np.random.choice([1, 2, 3])  # choose between 1 and 2
        for _ in range(error_order):
            circuit.add_gate('X', i)
        print(f'X^{error_order} error inserted on qubit {i}')
    else:
        print('No error inserted')
    return
insert_ququart_x_error(ququart_circuit, p=0.8)

X^3 error inserted on qubit 2


In [12]:
# ZZ^3I
ququart_circuit.add_gate('CNOT', 0, 3)
ququart_circuit.add_gate('CNOT', 1, 3)
ququart_circuit.add_gate('CNOT', 1, 3)
ququart_circuit.add_gate('CNOT', 1, 3)
# IZZ^3
ququart_circuit.add_gate('CNOT', 1, 4)
ququart_circuit.add_gate('CNOT', 2, 4)
ququart_circuit.add_gate('CNOT', 2, 4)
ququart_circuit.add_gate('CNOT', 2, 4)
# Measure
ququart_circuit.add_gate('MEASURE', 3)
ququart_circuit.add_gate('MEASURE', 4)

ququart_program = Program(ququart_circuit)
ququart_result = ququart_program.simulate()
print(ququart_result)

[Measured qudit (3) as (0) and was deterministic, Measured qudit (4) as (0) and was deterministic]


In [13]:
cirq_ququart_circuit = circuit_to_cirq_circuit(ququart_circuit)
print(cirq_ququart_circuit)

                                              ┌─────────────────┐   ┌─────────────────┐   ┌─────────────────┐                    ┌─────────────────┐
0 (d=3): ───CNOT_3_control───CNOT_3_control───────CNOT_3_control────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
            │                │                    │
1 (d=3): ───CNOT_3_target────┼────────────────────┼─────────────────────CNOT_3_control────────CNOT_3_control────CNOT_3_control────CNOT_3_control────────────────────────────────────────────────────────────────
                             │                    │                     │                     │                 │                 │
2 (d=3): ────────────────────CNOT_3_target─────X_3┼──────────────────X_3┼──────────────────X_3┼─────────────────┼─────────────────┼────────────────────CNOT_3_control───CNOT_3_control───CNOT_3_control─────────
                                        